# 编译 XS-GEM5  Complie XS-GEM5

这里我们将展示XS-GEM5的编译流程

Here we will demonstrate the compilation process of XS-GEM5

XS-GEM5 使用 DRAMsim3 作为内存模拟器，在编译运行 XS-GEM5 之前需要先编译 DRAMsim3。

XS-GEM5 uses DRAMsim3 as its memory simulator.

Before compiling and running XS-GEM5, we need to compile DRAMsim3 first.

In [ ]:
%%bash
cd .. && source env.sh
pwd
who am i
cd ${GEM5_HOME}/ext/dramsim3

if [ ! -d "DRAMsim3" ]; then
    git clone https://github.com/umd-memsys/DRAMSim3.git DRAMsim3
fi

mkdir -p DRAMsim3/build
cd DRAMsim3/build
cmake ..
make -j `nproc`

等待编译完毕会获得编译好的Dramsim3，是我们接下来实验的重要部分。


Wait for the compilation to finish, and we will obtain the compiled Dramsim3, which is an important component for our upcoming experiments.

随后可以使用 scons 编译 GEM5。

`--linker=mold` 是可选的，可以加速编译过程，需要事先安装 mold 链接器，我们提供的 code-server 镜像中已经包含。

以上命令最后会生成 gem5.opt文件。

Then you can use scons to compile GEM5.

The option --linker=mold is optional and can speed up the compilation process
You need to install the mold linker beforehand, but it is already included in our code-server image.

The above command will generate the gem5.opt file.

In [ ]:
%%bash
cd .. && source env.sh
cd ${GEM5_HOME}

scons build/RISCV/gem5.opt --linker=mold -j8 `nproc`